In [1]:
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

In [2]:
housing = fetch_california_housing()
housing.data.shape

(20640, 8)

In [3]:
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [4]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler  = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test  = scaler.transform(X_test)

X_train.shape, X_valid.shape, X_test.shape

((11610, 8), (3870, 8), (5160, 8))

In [5]:
input_A = keras.layers.Input(shape = [5], name = "wide_input")
input_B = keras.layers.Input(shape = [6], name = "deep_input")
hidden1 = keras.layers.Dense(30, activation = "relu")(input_B)
hidden2 = keras.layers.Dense(30, activation = "relu")(hidden1)
concat  = keras.layers.concatenate([input_A, hidden2])
output  = keras.layers.Dense(1, name = "output")(concat)
model   = keras.Model(inputs = [input_A, input_B], outputs = [output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           210         deep_input[0][0]                 
__________________________________________________________________________________________________
wide_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           930         dense[0][0]                      
______________________________________________________________________________________________

In [6]:
model.compile(loss = "mse", optimizer = keras.optimizers.SGD(lr = 1e-3))

X_train_A, X_train_B = X_train[:, : 5], X_train[:, -6 :]
X_valid_A, X_valid_B = X_valid[:, : 5], X_valid[:, -6 :]
X_test_A , X_test_B  = X_test[:, : 5] , X_test[:, -6 :]
X_train_A.shape, X_train_B.shape

((11610, 5), (11610, 6))

In [14]:
checkpoint_cb     = keras.callbacks.ModelCheckpoint("./checkpoint/my_keras_model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

history  = model.fit((X_train_A, X_train_B), y_train, validation_data = ((X_valid_A, X_valid_B), y_valid),\
                     epochs = 100, verbose = 1, callbacks = [checkpoint_cb, early_stopping_cb])

mse_test = model.evaluate((X_test_A, X_test_B), y_test)

Epoch 1/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3609 - val_loss: 0.3436
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3606 - val_loss: 0.3422
Epoch 3/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3601 - val_loss: 0.3436
Epoch 4/100
363/363 [==============================] - 1s 3ms/step - loss: 0.3598 - val_loss: 0.3435
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3597 - val_loss: 0.3419
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3594 - val_loss: 0.3426
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3591 - val_loss: 0.3490
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3588 - val_loss: 0.3423
Epoch 9/100
363/363 [==============================] - 1s 4ms/step - loss: 0.3586 - val_loss: 0.3444
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3584 - val_lo